### Imports

In [1]:
import json
import csv
import pandas as pd
import requests
import matplotlib.pyplot as plt
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go

# Notebook - Uso das Espécies

## Funções Auxiliares

In [2]:
def create_list_of_items(dictionary: dict) -> list:
    """
    Converts a dictionary into a list of item pairs, where each key is paired with each of its values.
    
    Args:
        dictionary (dict): A dictionary where each key maps to a list of values.
        
    Returns:
        list: A list of lists, where each inner list contains a key-value pair from the dictionary.
    """
    rows = []
    for key, values in dictionary.items():
        for value in values:
            rows.append([key, value])
    return rows

## Leitura dos Assessements

In [3]:
def json_to_dataframe(json_file: str) -> pd.DataFrame:
    """
    Reads a JSON file and converts it into a Pandas DataFrame.
    
    Args:
        json_file (str): The path to the JSON file where each line is a JSON object.
        
    Returns:
        pd.DataFrame: A DataFrame containing the structured data from the JSON file.
    """
    data = []
    with open(json_file, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    dataframe = pd.json_normalize(data)
    return dataframe

In [4]:
json_file = 'assessements.json'
dataframe = json_to_dataframe(json_file)

## Renomeia e Agrupa Usos

In [5]:
def rename_uses(df: pd.DataFrame) -> None:
    """
    Renames and standardizes values in the 'use_and_trade' column of a DataFrame.
    
    This function processes a DataFrame to unify different terms in the 'use_and_trade' column, consolidating them into standard categories.
    Unknown or unlisted uses are labeled appropriately.
    
    Args:
        df (pd.DataFrame): The DataFrame containing a 'use_and_trade' column, with each entry being a list of uses.
        
    Returns:
        None: The function modifies the DataFrame in place.
    """
    valid_uses = ['Pets/display animals, horticulture', 
                  'Sport hunting/specimen collecting', 
                  'Construction or structural materials', 
                  'Fuels',
                  'Medicine - human & veterinary', 
                  'Research',
                  'Handicrafts, jewellery, etc.',
                  'Wearing apparel, accessories']
    for i in range(len(df)):
        uses = df.iloc[i]['use_and_trade']
        new_uses = []
        for use in uses:
            if use == "Unknown" and len(uses) <= 1:
                new_uses = ["Unknown"]
            elif use in valid_uses:
                new_uses.append(use)
            elif use == "Food - human" or use == "Food - animal":
                if "Food" not in new_uses:
                    new_uses.append("Food")
            elif use == 'Manufacturing chemicals' or use == 'Other chemicals':
                if 'Chemicals' not in new_uses:
                    new_uses.append('Chemicals')
            else:
                if 'Others' not in new_uses:
                    new_uses.append('Others')
        if len(uses) == 0:
            new_uses = ["Unknown"]
        df.at[i, 'use_and_trade'] = new_uses

In [6]:
def renomeia_usos(df):
    valid_uses = {'Pets/display animals, horticulture': 'Pets e Horticultura', 
                  'Sport hunting/specimen collecting': 'Caça', 
                  'Construction or structural materials': 'Materiais de construção', 
                  'Fuels': 'Combustíveis',
                  'Medicine - human & veterinary': 'Medicina', 
                  'Research': 'Pesquisa',
                  'Handicrafts, jewellery, etc.': 'Artesanato, joias, etc',
                  'Wearing apparel, accessories': 'Roupas, acessórios'}
    for i in range(len(df)):
        uses = df.iloc[i]['use_and_trade']
        new_uses = []
        for use in uses:
            if use == "Unknown" and len(uses) <= 1:
                new_uses = ["Desconhecido"]
            elif use in valid_uses:
                new_uses.append(valid_uses[use])
            elif use == "Food - human" or use == "Food - animal":
                if "Alimentação" not in new_uses:
                    new_uses.append("Alimentação")
            elif use == 'Manufacturing chemicals' or use == 'Other chemicals':
                if 'Químicos' not in new_uses:
                    new_uses.append('Químicos')
            else:
                if 'Outros' not in new_uses:
                    new_uses.append('Outros')
        if len(uses) == 0:
            new_uses = ["Desconhecido"]
        df.at[i, 'use_and_trade'] = new_uses

In [7]:
renomeia_usos(dataframe)
display(dataframe)

,locations,red_list_category,threats,use_and_trade,year_published,taxon.class_name,taxon.family_name,taxon.kingdom_name,taxon.order_name,taxon.phylum_name,taxon.scientific_name,taxon.sis_id
0,"[{'country': 'Zimbabwe', 'presence': 'Extant'}...",LC,[],[Desconhecido],2019,MAGNOLIOPSIDA,BURSERACEAE,PLANTAE,SAPINDALES,TRACHEOPHYTA,Commiphora glandulosa,146223165
1,"[{'country': 'Peru', 'presence': 'Extant'}, {'...",LC,[],[Pets e Horticultura],2018,AVES,ICTERIDAE,ANIMALIA,PASSERIFORMES,CHORDATA,Dives warczewiczi,22724305
2,"[{'country': 'Paraguay', 'presence': 'Extant'}...",NR,[],[Desconhecido],2000,AVES,BUCCONIDAE,ANIMALIA,PICIFORMES,CHORDATA,Notharchus swainsoni,22733165
3,"[{'country': 'Togo', 'presence': 'Extant'}, {'...",LC,[],[Alimentação],2015,ACTINOPTERYGII,MUGILIDAE,ANIMALIA,MUGILIFORMES,CHORDATA,Mugil curema,190168
4,"[{'country': 'Ogasawara-shoto', 'presence': 'E...",DD,[],[Desconhecido],1996,GASTROPODA,CAMAENIDAE,ANIMALIA,STYLOMMATOPHORA,MOLLUSCA,Mandarina aureola,12741
...,...,...,...,...,...,...,...,...,...,...,...,...
320979,"[{'country': 'Rondônia', 'presence': 'Extant'}...",EN,"[Agro-industry grazing, ranching or farming, U...",[Desconhecido],2021,ACTINOPTERYGII,RIVULIDAE,ANIMALIA,CYPRINODONTIFORMES,CHORDATA,Anablepsoides luitalimae,173823654
320980,"[{'country': 'Sicilia', 'presence': 'Extant'},...",EN,"[Recreational activities, Tourism & recreation...",[Desconhecido],2016,INSECTA,ACRIDIDAE,ANIMALIA,ORTHOPTERA,ARTHROPODA,Sphingonotus personatus,16084535
320981,"[{'country': 'Solomon Islands', 'presence': 'E...",LC,[],"[Artesanato, joias, etc, Caça]",2013,GASTROPODA,CONIDAE,ANIMALIA,NEOGASTROPODA,MOLLUSCA,Conus biliosus,192420
320982,[],E,[],[Desconhecido],1986,MAMMALIA,CERCOPITHECIDAE,ANIMALIA,PRIMATES,CHORDATA,Mandrillus leucophaeus,12753


## Filtrar DataFrame

In [8]:
def create_list_unique_years(dataframe: pd.DataFrame) -> list:
    """
    Extracts and returns a sorted list of unique years from the 'year_published' column in the DataFrame.
    
    Args:
        dataframe (pd.DataFrame): The DataFrame containing the 'year_published' column with publication years.
        
    Returns:
        list: A sorted list of unique years as integers, excluding any non-integer values.
    """
    years = list(dataframe['year_published'].unique())
    unique_years = []
    for value in years:
        try:
            int_value = int(value)
            unique_years.append(int_value)
        except:
            continue
    unique_years.sort()
    return unique_years

### Listas de IDs únicos e Anos únicos

In [9]:
unique_ids = list(dataframe['taxon.sis_id'].unique())
unique_years = create_list_unique_years(dataframe)

In [10]:
def create_dict_uses_by_id(dataframe: pd.DataFrame, column: str, unique_ids: list) -> dict:
    """
    Creates a dictionary mapping each unique taxon ID to the longest list of values in a specified column.
    
    Args:
        dataframe (pd.DataFrame): The DataFrame containing species data and the specified column.
        column (str): The name of the column to retrieve lists of values from.
        unique_ids (list): A list of unique taxon IDs (from 'taxon.sis_id') to filter by.
        
    Returns:
        dict: A dictionary where each key is a taxon ID, and each value is the longest list of values from the specified column.
    """
    dict_uses_id = {}
    for i in range(len(unique_ids)):
        filtered_dataframe = dataframe[dataframe['taxon.sis_id'] == unique_ids[i]]
        list_of_max_length = []
        for j in range(len(filtered_dataframe)):
            list_of_values = filtered_dataframe.iloc[j][column]
            if len(list_of_values) > len(list_of_max_length):
                list_of_max_length = list_of_values
        dict_uses_id[unique_ids[i]] = list_of_max_length
    return dict_uses_id

### Dataframe de Usos por ID

In [11]:
def create_dataframe_uses_by_id(dataframe: pd.DataFrame, unique_ids: list) -> pd.DataFrame:
    """
    Creates a DataFrame that maps each taxon ID to its associated uses or trades.
    
    Args:
        dataframe (pd.DataFrame): The DataFrame containing species data, including the 'use_and_trade' column.
        unique_ids (list): A list of unique taxon IDs to filter by.
        
    Returns:
        pd.DataFrame: A DataFrame with columns "ID" and "Use", representing each taxon ID and its associated uses or trades.
    """
    dict_uses_id = create_dict_uses_by_id(dataframe, 'use_and_trade', unique_ids)
    rows = create_list_of_items(dict_uses_id)
    return pd.DataFrame(rows, columns=["ID", "Use"])

In [12]:
uses_dataframe = create_dataframe_uses_by_id(dataframe, unique_ids)
display(uses_dataframe)

,ID,Use
0,146223165,Desconhecido
1,22724305,Pets e Horticultura
2,22733165,Desconhecido
3,190168,Alimentação
4,12741,Desconhecido
...,...,...
191691,194012578,Desconhecido
191692,173823654,Desconhecido
191693,16084535,Desconhecido
191694,192420,"Artesanato, joias, etc"


**Salvar Dataframe**

In [ ]:
uses_dataframe.to_csv("usos.csv")

**Abrir Dataframe**

In [ ]:
uses_dataframe = pd.read_csv("usos.csv")

### Dataframe de IDs por País

In [13]:
def create_dataframe_countries_by_id(dataframe: pd.DataFrame, unique_ids: list) -> pd.DataFrame:
    """
    Creates a DataFrame mapping each taxon ID to the countries in which it is located.
    
    Args:
        dataframe (pd.DataFrame): The DataFrame containing species data, including the 'locations' column.
        unique_ids (list): A list of unique taxon IDs to filter by.
        
    Returns:
        pd.DataFrame: A DataFrame with columns "ID" and "Country", where each row represents a taxon ID and a country where it is found.
    """
    dict_countries_id = create_dict_uses_by_id(dataframe, 'locations', unique_ids)
    rows = []
    for key, values in dict_countries_id.items():
        for value in values:
            rows.append([key, value['country']])
    return pd.DataFrame(rows, columns=["ID", "Country"])

In [14]:
countries_dataframe = create_dataframe_countries_by_id(dataframe, unique_ids)
display(countries_dataframe)

,ID,Country
0,146223165,Zimbabwe
1,146223165,Zambia
2,146223165,"Congo, The Democratic Republic of the"
3,146223165,Angola
4,146223165,Mozambique
...,...,...
1206783,192420,Ashmore-Cartier Is.
1206784,192420,Comoros
1206785,192420,Australia
1206786,192420,Coral Sea Is. Territory


**Lista única de países**

In [15]:
countries = list(countries_dataframe["Country"].unique())

**Salvar dataframe**

In [28]:
countries_dataframe.to_csv("paises.csv")

**Carregar dataframe**

In [ ]:
countries_dataframe = pd.read_csv("paises.csv")

In [16]:
def filter_uses_by_country(uses_dataframe: pd.DataFrame, countries_dataframe: pd.DataFrame, country: str) -> dict:
    """
    Filters and counts the occurrences of uses within a specified country, excluding unknown uses.
    
    Args:
        uses_dataframe (pd.DataFrame): DataFrame containing 'ID' and 'Use' columns representing taxon IDs and associated uses.
        countries_dataframe (pd.DataFrame): DataFrame with 'ID' and 'Country' columns linking taxon IDs to countries.
        country (str): The name of the country to filter by.
        
    Returns:
        dict: A dictionary where keys are use types and values are their counts within the specified country.
    """
    list_ids_country = list(countries_dataframe[countries_dataframe['Country'] == country]["ID"])
    filtered_df = uses_dataframe[uses_dataframe['ID'].isin(list_ids_country)]
    usage_counts = filtered_df['Use'].value_counts().to_dict()
    del usage_counts['Desconhecido']
    return usage_counts

In [26]:
country = "Montenegro"
usage_counts = filter_uses_by_country(uses_dataframe, countries_dataframe, country)

### Categorias de Risco Únicas

In [88]:
unique_categories = list(dataframe["red_list_category"].dropna().unique())

In [89]:
print(unique_categories)

['LC', 'NR', 'DD', 'EN', 'VU', 'CR', 'NT', 'LR/lc', 'V', 'I', 'R', 'NA', 'T', 'EX', 'EW', 'K', 'LR/nt', 'NE', 'N/A', 'Ex', 'LR/cd', 'E', 'RE', 'nt', 'Ex?', 'Ex/E', 'O', 'CUSTOM', 'CT']


### Filtrar Dataframe por Anos

In [17]:
def filter_some_years(dataframe: pd.Dataframe, list_years: list) -> pd.DataFrame:
    """
    Filters the DataFrame to include only entries published in specified years.
    
    Args:
        dataframe (pd.DataFrame): DataFrame containing a 'year_published' column with publication years.
        list_years (list): List of years to filter the data by.
        
    Returns:
        pd.DataFrame: Filtered DataFrame containing only rows with publication years in list_years.
    """
    return dataframe[dataframe['year_published'].isin(list_years)]

In [18]:
def count_uses_accumulated_years(dataframe: pd.DataFrame, uses_dataframe: pd.DataFrame, list_years: list) -> dict:
    """
    Counts the occurrences of different uses over a specified range of years, excluding unknown uses.
    
    Args:
        dataframe (pd.DataFrame): DataFrame containing 'year_published' and 'taxon.sis_id' columns.
        uses_dataframe (pd.DataFrame): DataFrame containing 'ID' and 'Use' columns for taxon IDs and their uses.
        list_years (list): List of years to accumulate use counts over.
        
    Returns:
        dict: A dictionary where keys are use types and values are accumulated counts across the specified years.
    """
    years_dataframe = filter_some_years(dataframe, list_years)
    unique_ids = list(years_dataframe['taxon.sis_id'].unique())
    filtered_dataframe = uses_dataframe[uses_dataframe['ID'].isin(unique_ids)]
    usage_counts = filtered_dataframe['Use'].value_counts().to_dict()
    del usage_counts['Desconhecido']
    return usage_counts

In [204]:
usage = count_uses_accumulated_years(dataframe, uses_dataframe, ['2020'])

## Plotar Gráficos

In [237]:
# Inicialização do app Dash
app = dash.Dash(__name__)

# Layout do app com dropdowns sequenciais
app.layout = html.Div([
    html.H1("Gráfico Interativo de Uso das Espécies", style={"text-align": "center", "text-decoration": "underline wavy black"}),
    
    # Dropdown para seleção de país (múltipla escolha)
    
    html.Div([
        html.H3("Filtragem por País"),
        dcc.Dropdown(
            id="country-dropdown",
            options=[{"label": c, "value": c} for c in countries],
            placeholder="Selecione um País",
            multi=True
        ),
    ], style={'margin-bottom': '20px', "padding": "10px", "padding-top": "5px", "border": "1px solid #ddd", "border-radius": "5px"}),
    
    html.Div([
        html.H3("Filtragem por Taxonomia"),
        dcc.Dropdown(id="kingdom-dropdown", placeholder="Selecione um Reino", style={"margin-bottom": "10px"}),
        dcc.Dropdown(id="phylum-dropdown", placeholder="Selecione um Filo", style={"margin-bottom": "10px"}),
        dcc.Dropdown(id="class-dropdown", placeholder="Selecione uma Classe", style={"margin-bottom": "10px"}),
        dcc.Dropdown(id="order-dropdown", placeholder="Selecione uma Ordem", style={"margin-bottom": "10px"}),
        dcc.Dropdown(id="family-dropdown", placeholder="Selecione uma Família", style={"margin-bottom": "10px"}),
        dcc.Dropdown(id="specie-dropdown", placeholder="Selecione uma Espécie", style={"margin-bottom": "10px"}),
    ], style={'margin-bottom': '20px', "padding": "10px", "padding-top": "5px", "border": "1px solid #ddd", "border-radius": "5px"}),
    
    # Dropdown para seleção de ano (múltipla escolha)
    html.Div([
        html.H3("Filtragem por Ano"),
        dcc.Dropdown(
            id="year-dropdown",
            options=[{"label": year, "value": year} for year in unique_years],
            placeholder="Selecione um ou mais Anos",
            multi=True
        ),
    ], style={'margin-bottom': '20px', "padding": "10px", "padding-top": "5px", "border": "1px solid #ddd", "border-radius": "5px"}),
    
    # Título das opções de visualização
    html.Div([
        html.H3("Opções de Visualização"),
        dcc.Checklist(
                id="default-mode-checklist",
                options=[{"label": "Gráfico acumulado", "value": "default_mode"}],
                value=["default_mode"],
                labelStyle={'display': 'block', "margin-bottom": "5px"}
        ),
        dcc.Checklist(
            id="country-mode-checklist",
            options=[{"label": "Gráfico empilhado por país", "value": "country_mode"}],
            labelStyle={"display": "block", "margin-bottom": "5px"}
        ),
        dcc.Checklist(
            id="year-mode-checklist",
            options=[{"label": "Gráfico empilhado por ano", "value": "year_mode"}],
            labelStyle={"display": "block", "margin-bottom": "5px"}
        ), 
        dcc.Checklist(
            id="category-checklist",
            options=[{"label": "Gráfico empilhado por categoria de risco", "value": "category_mode"}],
            labelStyle={"display": "block", "margin-bottom": "5px"}
        ),
        
    ], style={"margin-top": "20px", "padding": "10px", "padding-top": "5px", "border": "1px solid #ddd", "border-radius": "5px"}),

    html.Div([
        html.H3("Opções de Valores"),
        dcc.Checklist(
                id="absolute-mode-checklist",
                options=[{"label": "Valores Absolutos", "value": "absolute_mode"}],
                value=["absolute-mode"],
                labelStyle={'display': 'block', "margin-bottom": "5px"}
        ),
        dcc.Checklist(
                id="percentage-mode-checklist",
                options=[{"label": "Valores Percentuais", "value": "percentage_mode"}],
                labelStyle={'display': 'block', "margin-bottom": "5px"}
        )
    ], style={"margin-top": "20px", 'margin-bottom': '20px', "padding": "10px", "padding-top": "5px", "border": "1px solid #ddd", "border-radius": "5px"}),

    # Gráfico de barras
    dcc.Graph(id="stacked-bar-chart")
], style={"width": "95%", "margin": "0 auto"})


In [238]:
def update_fig_layout(fig, title: str, x_label: str, y_label: str) -> None:
    """
    Updates the layout of a Plotly figure with a title, axis labels, and a specified theme.
    
    Args:
        fig (plotly.graph_objs.Figure): The figure to update.
        title (str): The title of the plot.
        x_label (str): Label for the x-axis.
        y_label (str): Label for the y-axis.
        
    Returns:
        None: Modifies the figure in place.
    """
    fig.update_layout(
            title=title,
            xaxis_title=x_label,
            yaxis=dict(title=y_label, categoryorder='total ascending'),
            barmode='stack',
            template="plotly_white"
        )

In [239]:
def add_bar(fig, x_values: list, y_values: list, name: str) -> None:
    """
    Adds a horizontal bar to a Plotly figure.
    
    Args:
        fig (plotly.graph_objs.Figure): The figure to which the bar will be added.
        x_values (list): Values along the x-axis.
        y_values (list): Values along the y-axis.
        name (str): The name of the bar trace.
        
    Returns:
        None: Modifies the figure in place.
    """
    fig.add_trace(go.Bar(
                x=x_values,
                y=y_values,
                name=name,
                orientation='h'
            ))

In [240]:
def calculate_values_per_mode(list_of_values: list, total: int, percentage_mode: bool) -> list:
    """
    Calculates values as percentages or raw values based on a mode.
    
    Args:
        list_of_values (list): A list of numerical values to process.
        total (int or float): The total value used to calculate percentages.
        percentage_mode (bool): If True, returns values as percentages of total; otherwise, returns raw values.
        
    Returns:
        list: A list of values, either as percentages or as raw values depending on the mode.
    """
    if percentage_mode:
        return [(value / total) * 100 if total > 0 else 0 for value in list_of_values]
    return list_of_values

In [241]:
def filter_taxonomy(dataframe: pd.DataFrame, 
                   selected_kingdom: str, 
                   selected_phylum: str, 
                   selected_class: str, 
                   selected_order: str, 
                   selected_family: str, 
                   selected_specie: str) -> pd.DataFrame:
    """
    Filters the DataFrame based on selected taxonomic criteria.
    
    Args:
        dataframe (pd.DataFrame): DataFrame containing taxonomic columns such as 'taxon.kingdom_name', 'taxon.phylum_name', etc.
        selected_kingdom (str): The kingdom to filter by, or None to skip filtering by kingdom.
        selected_phylum (str): The phylum to filter by, or None to skip filtering by phylum.
        selected_class (str): The class to filter by, or None to skip filtering by class.
        selected_order (str): The order to filter by, or None to skip filtering by order.
        selected_family (str): The family to filter by, or None to skip filtering by family.
        selected_specie (str): The species to filter by, or None to skip filtering by species.
        
    Returns:
        pd.DataFrame: A filtered DataFrame containing only entries matching the selected taxonomic criteria.
    """
    filtered_df = dataframe.copy()
    if selected_kingdom:
        filtered_df = dataframe[dataframe["taxon.kingdom_name"] == selected_kingdom]
    if selected_phylum:
        filtered_df = filtered_df[filtered_df["taxon.phylum_name"] == selected_phylum]
    if selected_class:
        filtered_df = filtered_df[filtered_df["taxon.class_name"] == selected_class]
    if selected_order:
        filtered_df = filtered_df[filtered_df["taxon.order_name"] == selected_order]
    if selected_family:
        filtered_df = filtered_df[filtered_df["taxon.family_name"] == selected_family]
    if selected_specie:
        filtered_df = filtered_df[filtered_df["taxon.scientific_name"] == selected_specie]
    return filtered_df

In [242]:
def generate_uses_count(dataframe: pd.DataFrame, uses_dataframe: pd.DataFrame) -> dict:
    ids = list(dataframe['taxon.sis_id'].unique())
    filtered_dataframe = uses_dataframe[uses_dataframe['ID'].isin(ids)]
    usage_counts = filtered_dataframe['Use'].value_counts().to_dict()
    try:
        del usage_counts['Desconhecido']
    except:
        pass
    return usage_counts

In [243]:
def create_bars(fig, all_uses: list, dict_uses: dict, list_selected_items: list, total: int, percentage_mode: bool):
    for item in list_selected_items:
        #print(item)
        usage_counts = dict_uses[item]
        values = [usage_counts.get(use, 0) for use in all_uses]
        x_list = calculate_values_per_mode(values, total, percentage_mode)
        add_bar(fig, x_list, all_uses, item)

In [244]:
def create_figure_with_bar(dict_counts: dict, percentage_mode: bool):
    categories = list(dict_counts.keys())
    values = list(dict_counts.values())
    values = calculate_values_per_mode(values, sum(values), percentage_mode)
    return go.Figure(go.Bar(
        x=values,
        y=categories,
        orientation='h',
        marker=dict(color='skyblue')
    ))

In [245]:
@app.callback(
    [Output("default-mode-checklist", "value"), Output("country-mode-checklist", "value"), Output("year-mode-checklist", "value"), Output("category-checklist", "value")],
    [Input("default-mode-checklist", "value"), Input("country-mode-checklist", "value"), Input("year-mode-checklist", "value"), Input("category-checklist", "value")],
)
def update_mode_checklists(selected_default_mode, selected_country_mode, selected_year_mode, selected_category_mode):
    ctx = dash.callback_context  # contexto do callback para identificar o acionador

    #if not ctx.triggered:
    #    return [], [], []  # Nenhum checklist acionado, retorna ambos vazios

    triggered_id = ctx.triggered[0]["prop_id"].split(".")[0]
    
    #if not any([selected_country_mode, selected_year_mode, selected_category_mode]):
        #return ["default_mode"], [], [], []
    
    #if triggered_id == "default-mode-checklist" and if selected_default_mode:
        #return ["default_mode"], [], [], []
        #else:
            #return ["default_mode"], [], [], []

    # Se o checklist de país foi acionado, ajusta para "stacked_country" e limpa o de ano
    if triggered_id == "country-mode-checklist" and selected_country_mode:  # se estiver selecionado, aplica "stacked_country"
        return [], ["country_mode"], [], []
        #else:  # se desmarcado, mantém ambos desmarcados
            #return ["default_mode"], [], [], []

    # Se o checklist de ano foi acionado, ajusta para "stacked_year" e limpa o de país
    elif triggered_id == "year-mode-checklist" and selected_year_mode:  # se estiver selecionado, aplica "stacked_year"
        return [], [], ["year_mode"], []
        #else:  # se desmarcado, mantém ambos desmarcados
            #return ["default_mode"], [], [], []
        
    elif triggered_id == "category-checklist" and selected_category_mode:  # se estiver selecionado, aplica "stacked_year"
        return [], [], [], ["category_mode"]
        #else:  # se desmarcado, mantém ambos desmarcados
        #    return ["default_mode"], [], [], [] 
    return ["default_mode"], [], [], []

@app.callback(
    [Output("absolute-mode-checklist", "value"),
     Output("percentage-mode-checklist", "value")],
    [Input("absolute-mode-checklist", "value"),
     Input("percentage-mode-checklist", "value")]
)
def update_values_mode_checklists(absolute_value, percentage_value):
    ctx = dash.callback_context
    
    if not ctx.triggered:
        # Inicia com o modo padrão marcado
        return ["absolute_mode"], []

    triggered_id = ctx.triggered[0]["prop_id"].split(".")[0]

    if triggered_id == "absolute-mode-checklist" and absolute_value:
        return ["absolute_mode"], []

    elif triggered_id == "percentage-mode-checklist" and percentage_value:
        return [], ["percentage_mode"]

    return ["absolute_mode"], []  # Caso todos sejam desmarcados
        
# Callback para atualizar o dropdown de reino baseado no país selecionado
@app.callback(
    Output("kingdom-dropdown", "options"),
    [Input("country-dropdown", "value")]
)
def update_kingdom_options(selected_countries):
    #if not selected_countries:
        #return []
    kingdoms = dataframe["taxon.kingdom_name"].unique()
    return [{"label": k, "value": k} for k in kingdoms]

# Callbacks para atualizar os dropdowns sequencialmente (reino, filo, classe, ordem, família)
@app.callback(
    Output("phylum-dropdown", "options"),
    [Input("kingdom-dropdown", "value"), Input("country-dropdown", "value")]
)
def update_phylum_options(selected_kingdom, selected_countries):
    if not selected_kingdom:
        return []
    phyla = dataframe[dataframe["taxon.kingdom_name"] == selected_kingdom]["taxon.phylum_name"].unique()
    return [{"label": p, "value": p} for p in phyla]

@app.callback(
    Output("class-dropdown", "options"),
    [Input("phylum-dropdown", "value"), Input("kingdom-dropdown", "value")]
)
def update_class_options(selected_phylum, selected_kingdom):
    if selected_phylum is None:
        return []
    classes = dataframe[(dataframe["taxon.kingdom_name"] == selected_kingdom) & 
                 (dataframe["taxon.phylum_name"] == selected_phylum)]["taxon.class_name"].unique()
    return [{"label": c, "value": c} for c in classes]

@app.callback(
    Output("order-dropdown", "options"),
    [Input("class-dropdown", "value"), Input("phylum-dropdown", "value"), Input("kingdom-dropdown", "value")]
)
def update_order_options(selected_class, selected_phylum, selected_kingdom):
    if selected_class is None:
        return []
    orders = dataframe[(dataframe["taxon.kingdom_name"] == selected_kingdom) & 
                (dataframe["taxon.phylum_name"] == selected_phylum) &
                (dataframe["taxon.class_name"] == selected_class)]["taxon.order_name"].unique()
    return [{"label": o, "value": o} for o in orders]

@app.callback(
    Output("family-dropdown", "options"),
    [Input("order-dropdown", "value"), Input("class-dropdown", "value"), Input("phylum-dropdown", "value"), Input("kingdom-dropdown", "value")]
)
def update_family_options(selected_order, selected_class, selected_phylum, selected_kingdom):
    if selected_order is None:
        return []
    families = dataframe[(dataframe["taxon.kingdom_name"] == selected_kingdom) & 
                  (dataframe["taxon.phylum_name"] == selected_phylum) &
                  (dataframe["taxon.class_name"] == selected_class) &
                  (dataframe["taxon.order_name"] == selected_order)]["taxon.family_name"].unique()
    return [{"label": f, "value": f} for f in families]

@app.callback(
    Output("specie-dropdown", "options"),
    [Input("family-dropdown", "value"), Input("order-dropdown", "value"), Input("class-dropdown", "value"), Input("phylum-dropdown", "value"), Input("kingdom-dropdown", "value")]
)
def update_specie_options(selected_family, selected_order, selected_class, selected_phylum, selected_kingdom):
    if selected_family is None:
        return []
    species = dataframe[(dataframe["taxon.kingdom_name"] == selected_kingdom) & 
                  (dataframe["taxon.phylum_name"] == selected_phylum) &
                  (dataframe["taxon.class_name"] == selected_class) &
                  (dataframe["taxon.order_name"] == selected_order) &
                  (dataframe["taxon.family_name"] == selected_family)]["taxon.scientific_name"].unique()
    return [{"label": f, "value": f} for f in species]

def filter_years(dataframe, countries_dataframe, selected_specie, selected_family, selected_order, selected_class, selected_phylum, selected_kingdom, selected_countries):
    if selected_kingdom is None:
        ids_species = []
    if selected_phylum is None:
        ids_species = list(dataframe[dataframe["taxon.kingdom_name"] == selected_kingdom]["taxon.sis_id"].dropna().unique())
    elif selected_class is None:
        ids_species = list(dataframe[(dataframe["taxon.kingdom_name"] == selected_kingdom) & 
                  (dataframe["taxon.phylum_name"] == selected_phylum)]["taxon.sis_id"].dropna().unique())
    elif selected_order is None:
        ids_species = list(dataframe[(dataframe["taxon.kingdom_name"] == selected_kingdom) & 
                  (dataframe["taxon.phylum_name"] == selected_phylum) &
                  (dataframe["taxon.class_name"] == selected_class)]["taxon.sis_id"].dropna().unique())
    elif selected_family is None:
        ids_species = list(dataframe[(dataframe["taxon.kingdom_name"] == selected_kingdom) & 
                  (dataframe["taxon.phylum_name"] == selected_phylum) &
                  (dataframe["taxon.class_name"] == selected_class) &
                  (dataframe["taxon.order_name"] == selected_order)]["taxon.sis_id"].dropna().unique())
    elif selected_specie is None:
        ids_species = list(dataframe[(dataframe["taxon.kingdom_name"] == selected_kingdom) & 
                  (dataframe["taxon.phylum_name"] == selected_phylum) &
                  (dataframe["taxon.class_name"] == selected_class) &
                  (dataframe["taxon.order_name"] == selected_order) &
                  (dataframe["taxon.family_name"] == selected_family)]["taxon.sis_id"].dropna().unique())
    else:
        ids_species = list(dataframe[(dataframe["taxon.kingdom_name"] == selected_kingdom) & 
                  (dataframe["taxon.phylum_name"] == selected_phylum) &
                  (dataframe["taxon.class_name"] == selected_class) &
                  (dataframe["taxon.order_name"] == selected_order) &
                  (dataframe["taxon.family_name"] == selected_family) &
                  (dataframe["taxon.scientific_name"] == selected_specie)]["taxon.sis_id"].dropna().unique())
    #ids_countries = []
    if selected_countries:
        ids_countries = list(countries_dataframe[countries_dataframe['Country'].isin(selected_countries)]['ID'].dropna().unique())
    if selected_countries and selected_kingdom:    
        ids = list(set(ids_countries) & set(ids_species))
        species = list(dataframe[dataframe["taxon.sis_id"].isin(ids)]["year_published"].dropna().unique())
    elif selected_kingdom:
        species = list(dataframe[dataframe["taxon.sis_id"].isin(ids_species)]["year_published"].dropna().unique())
    elif selected_countries:
        species = list(dataframe[dataframe["taxon.sis_id"].isin(ids_countries)]["year_published"].dropna().unique())
    else: 
        species = list(dataframe["year_published"].dropna().unique())
        #print(species)
    species.sort()
    return species
    

@app.callback(
    Output("year-dropdown", "options"),
    [Input("specie-dropdown", "value"),
     Input("family-dropdown", "value"), Input("order-dropdown", "value"),
     Input("class-dropdown", "value"), Input("phylum-dropdown", "value"),
     Input("kingdom-dropdown", "value"), Input("country-dropdown", "value")]
)
def update_years_options(selected_specie, selected_family, selected_order, selected_class, selected_phylum, selected_kingdom, selected_countries):
    years = filter_years(dataframe, countries_dataframe, selected_specie, selected_family, selected_order, selected_class, selected_phylum, selected_kingdom, selected_countries)
    return [{"label": f, "value": f} for f in years]



In [246]:
# Callback para atualizar o gráfico de barras
@app.callback(
    Output("stacked-bar-chart", "figure"),
    [Input("specie-dropdown", "value"),
     Input("family-dropdown", "value"), Input("order-dropdown", "value"),
     Input("class-dropdown", "value"), Input("phylum-dropdown", "value"),
     Input("kingdom-dropdown", "value"), Input("country-dropdown", "value"),
     Input("year-dropdown", "value"), 
     Input("country-mode-checklist", "value"), Input("year-mode-checklist", "value"), 
     Input("percentage-mode-checklist", "value"), Input("category-checklist", "value")]
)
def update_graph(selected_specie, selected_family, selected_order, selected_class, selected_phylum, 
                 selected_kingdom, selected_countries, selected_years, country_mode, year_mode, percentage_mode, category_mode):
    filtered_df = dataframe.copy()
    fig = go.Figure()
    total_by_use = {'Alimentação': 0,
                    'Pets e Horticultura': 0,
                    'Medicina': 0,
                    'Outros': 0,
                    'Caça': 0,
                    'Materiais de construção': 0,
                    'Combustíveis': 0,
                    'Artesanato, joias, etc': 0,
                    'Químicos': 0,
                    'Roupas, acessórios': 0,
                    'Pesquisa': 0}
    total = 0
    all_uses = list(total_by_use.keys())
    if country_mode:
        if selected_countries is None:
            selected_countries = countries
        dict_country_uses = {}        
        for country in selected_countries:
            list_ids_country = list(countries_dataframe[countries_dataframe['Country'] == country]["ID"])
            filtered_df = filtered_df[filtered_df['taxon.sis_id'].isin(list_ids_country)]
            filtered_df = filter_taxonomy(filtered_df, selected_kingdom, selected_phylum, selected_class, selected_order, selected_family, selected_specie)
            if selected_years:
                years_dataframe = filter_some_years(filtered_df, selected_years)
                ids = list(years_dataframe['taxon.sis_id'].unique())
                filtered_df = filtered_df[filtered_df['taxon.sis_id'].isin(ids)]
            usage_counts = generate_uses_count(filtered_df, uses_dataframe)
            dict_country_uses[country] = usage_counts

            for use in usage_counts.keys():
                total_by_use[use] += usage_counts[use]
                total += usage_counts[use]

        # Calcular os percentuais para cada país
        create_bars(fig, all_uses, dict_country_uses, selected_countries, total, percentage_mode)
        if percentage_mode:
            update_fig_layout(fig, "Uso das Espécies por País", "Percentual (%)", "Categorias")
        else:
            update_fig_layout(fig, "Uso das Espécies por País", "Contagem", "Categorias")

    elif year_mode:
        dict_year_uses = {}
        
        if selected_years is None:
            selected_years = filter_years(dataframe, countries_dataframe, selected_specie, selected_family, selected_order, selected_class, selected_phylum, selected_kingdom, selected_countries)

        if selected_countries:
            ids = list(countries_dataframe[countries_dataframe['Country'].isin(selected_countries)]['ID'].unique())
            filtered_df = filtered_df[filtered_df["taxon.sis_id"].isin(ids)]
        filtered_df = filter_taxonomy(filtered_df, selected_kingdom, selected_phylum, selected_class, selected_order, selected_family, selected_specie)    
            
        for year in selected_years:
            years_dataframe = filter_some_years(filtered_df, [str(year)])
            ids = list(years_dataframe['taxon.sis_id'].unique())
            temp_df = filtered_df[filtered_df['taxon.sis_id'].isin(ids)]
            usage_counts = generate_uses_count(temp_df, uses_dataframe)
            dict_year_uses[year] = usage_counts
            for use in usage_counts.keys():
                total_by_use[use] += usage_counts[use]
                total += usage_counts[use]
        # Calcular os percentuais para cada país
        create_bars(fig, all_uses, dict_year_uses, selected_years, total, percentage_mode)

        if percentage_mode:
            update_fig_layout(fig, "Uso das Espécies por Ano", "Percentual (%)", "Categorias")
        else:
            update_fig_layout(fig, "Uso das Espécies por Ano", "Contagem", "Categorias")
    elif category_mode:
        if selected_countries:
            ids = list(countries_dataframe[countries_dataframe['Country'].isin(selected_countries)]['ID'].unique())
            filtered_df = filtered_df[filtered_df["taxon.sis_id"].isin(ids)]
        filtered_df = filter_taxonomy(filtered_df, selected_kingdom, selected_phylum, selected_class, selected_order, selected_family, selected_specie)
        if selected_years:
            years_dataframe = filter_some_years(filtered_df, selected_years)
            ids = list(years_dataframe['taxon.sis_id'].unique())
            filtered_df = filtered_df[filtered_df['taxon.sis_id'].isin(ids)]
            
        dict_categories = {}
        for category in unique_categories:
            temp_dataframe = filtered_df[filtered_df["red_list_category"] == category]
            usage_counts = generate_uses_count(temp_dataframe, uses_dataframe)
            dict_categories[category] = usage_counts
            
            for use in usage_counts.keys():
                total_by_use[use] += usage_counts[use]
                total += usage_counts[use]
                
        create_bars(fig, all_uses, dict_categories, unique_categories, total, percentage_mode)
        if percentage_mode:
            update_fig_layout(fig, "Uso das Espécies por Categoria de Risco", "Percentual (%)", "Categorias")
        else:
            update_fig_layout(fig, "Uso das Espécies por Categoria de Risco", "Contagem", "Categorias")
        
    else:   
        if selected_countries:
            ids = list(countries_dataframe[countries_dataframe['Country'].isin(selected_countries)]['ID'].unique())
            filtered_df = filtered_df[filtered_df["taxon.sis_id"].isin(ids)]
        filtered_df = filter_taxonomy(filtered_df, selected_kingdom, selected_phylum, selected_class, selected_order, selected_family, selected_specie)
        if selected_years:
            years_dataframe = filter_some_years(filtered_df, selected_years)
            ids = list(years_dataframe['taxon.sis_id'].unique())
            filtered_df = filtered_df[filtered_df['taxon.sis_id'].isin(ids)]

        # Contar o número de espécies por categoria de uso
        usage_counts = generate_uses_count(filtered_df, uses_dataframe)
        for use in usage_counts.keys():
            total_by_use[use] += usage_counts[use]
            total += usage_counts[use]

        fig = create_figure_with_bar(total_by_use, percentage_mode)
        
        if percentage_mode:
            update_fig_layout(fig, "Uso das Espécies", "Percentual (%)", "Categorias")
        else:
            update_fig_layout(fig, "Uso das Espécies", "Contagem", "Categorias")
    
    return fig

In [247]:
# Executa o aplicativo
if __name__ == "__main__":
    app.run_server(debug=True)